# 추천 시스템 (Recommender Systems)


## 데이터 준비


In [101]:
import pandas as pd
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization

In [4]:
#전처리된 리뷰데이터Json파일 데이터프레임으로 불러오기
review_data = pd.read_json('preprocessed_review_data.json')

In [6]:
review_data

,review_idx,prod_idx,prod_satis,ship_satis,select_options,buy_date,review_date,use_opt1,use_satis1,use_opt2,use_satis2,content,user_id,memberSrl
0,168489442,2865723582,5,5,05_꿀밤고구마 못난이2kg(2개 주문시 10kg 발송),05_꿀밤고구마 못난이2kg(2개 주문시 10kg 발송),2020-10-06 15:25:44,맛,맛있어요,신선도,신선해요,좋아요좋아요좋아요좋아요좋아요,sln****,101994422
1,168390782,2865723582,5,5,02_★추천★꿀고구마 정품 특상1kg (2개 주문시 3kg 발송),02_★추천★꿀고구마 정품 특상1kg (2개 주문시 3kg 발송),2020-10-06 07:45:30,맛,보통이에요,신선도,적당해요,잘 받았습니다많이 달진않았어요,kej****,5016629
2,168389334,2865723582,5,5,05_꿀밤고구마 못난이2kg(2개 주문시 10kg 발송),05_꿀밤고구마 못난이2kg(2개 주문시 10kg 발송),2020-10-06 07:31:52,맛,맛있어요,신선도,신선해요,배송이 빠르네요 ㅎㅎ.,k38**,298123
3,168344378,2865723582,5,5,03_꿀고구마 정품한입1kg (2개 주문시 5kg 발송),03_꿀고구마 정품한입1kg (2개 주문시 5kg 발송),2020-10-05 23:55:31,맛,맛있어요,신선도,신선해요,좋은 가격에 잘 샀어요 배송도 빠릅니다,hko*****,80235
4,168329002,2865723582,5,5,02_★추천★꿀고구마 정품 특상1kg (2개 주문시 3kg 발송),02_★추천★꿀고구마 정품 특상1kg (2개 주문시 3kg 발송),2020-10-05 23:04:41,맛,맛있어요,신선도,신선해요,저렴한데 맛있어요.. 재구매 합니다.~^^,gra*****,17873545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568705,161355262,3873456926,5,4,12.닭갈비볶음밥 1+ 오븐바 오리지널 1,12.닭갈비볶음밥 1+ 오븐바 오리지널 1,2020-09-08 14:19:03,맛,보통이에요,신선도,적당해요,저렴하고 배송빨라툐ㅡㅡㅡ,khy****,153566
568706,161333522,3873456926,5,5,05.후랑크 오리지널 70g 1+1+1+1,05.후랑크 오리지널 70g 1+1+1+1,2020-09-08 12:55:21,맛,맛있어요,신선도,신선해요,싸게 잘삿어요 배송도 빨라요,rnt******,22123157
568707,161220022,3873456926,5,5,01.핫썸머 빨간맛 패키지 (꼬치불닭+오븐바매콤+프리미엄청양),01.핫썸머 빨간맛 패키지 (꼬치불닭+오븐바매콤+프리미엄청양),2020-09-08 00:14:24,맛,맛있어요,신선도,신선해요,아직 먹어보진 않았지만 포장긋ㆍ,hjk*****,80018294
568709,161188722,3873456926,5,5,06.후랑크 치즈 70g 1+1+1,06.후랑크 치즈 70g 1+1+1,2020-09-07 22:50:13,맛,맛있어요,신선도,적당해요,옥수수 소세지 강추요~~~~너무 맛있어요,dms*****,21201529


In [5]:
prod_data = pd.read_json('preprocessed_prod_data.json')

In [7]:
prod_data

,prod_idx,image,title,category,price,stocks,prod_satis,prod_url,buy_count
0,2865723582,http://th4.tmon.kr/thumbs/image/f32/0e0/a5e/b5...,[컬러푸드] 선착순20%할인쿠폰+해남 후기갑 꿀고구마 중상1kg 정품\n한입/특상/...,"[식품·건강, 채소, 감자·고구마·당근·뿌리채소, 고구마]",6900,1,4.4,http://www.tmon.co.kr/deal/2865723582\n,106276
1,512851998,http://th1.tmon.kr/thumbs/image/c8d/da8/a8e/99...,[샐러드미인] 간편한 샐러드/베이글 외 33종,"[식품·건강, 채소, 상추·깻잎·쌈채소·샐러드, 샐러드]",900,1,4.5,http://www.tmon.co.kr/deal/512851998\n,61337
2,760761146,http://th3.tmon.kr/thumbs/image/89e/370/471/8e...,[팜에이트] 간편한 소포장부터 든든한 한끼 샐러드까지 총14종,"[식품·건강, 채소, 상추·깻잎·쌈채소·샐러드, 샐러드]",1300,1,4.4,http://www.tmon.co.kr/deal/760761146\n,37744
3,212815213,http://image.ticketmonster.co.kr/deals/2015/06...,달콤해서 한입에 반한\n반시고구마 말랭이,"[식품·건강, 채소, 감자·고구마·당근·뿌리채소, 고구마]",1250,1,4.3,http://www.tmon.co.kr/deal/212815213\n,27041
4,3030712126,http://th1.tmon.kr/thumbs/image/154/93f/57b/e1...,[봉팔형님] 밤꿀고구마 중 1kg 외 꿀고구마 4종 크기별 모음 / 2개 구매시 3...,"[식품·건강, 채소, 감자·고구마·당근·뿌리채소, 고구마]",8900,1,4.4,http://www.tmon.co.kr/deal/3030712126\n,22321
...,...,...,...,...,...,...,...,...,...
31522,1783465342,http://img4.tmon.kr/cdn3/deals/2020/09/06/1783...,마이닭 닭가슴살소세지 핫바 오리지널 70g 100팩,"[식품·건강, 정육·계란류, 닭·오리고기, 생닭]",87940,1,5.0,http://www.tmon.co.kr/deal/1783465342\n,1
31523,1761347862,http://img4.tmon.kr/cdn3/deals/2020/05/28/1761...,허닭 닭가슴살 큐브 통현미 100g 30팩,"[식품·건강, 정육·계란류, 닭·오리고기, 생닭]",57160,1,0.0,http://www.tmon.co.kr/deal/1761347862\n,1
31524,1716499562,http://th1.tmon.kr/thumbs/image/b59/115/664/35...,[미트앤] 차돌양지국거리 500g,"[식품·건강, 정육·계란류, 수입육·기타, 미국산]",9000,1,0.0,http://www.tmon.co.kr/deal/1716499562\n,1
31525,1648136462,http://img3.tmon.kr/cdn3/deals/2019/05/29/1648...,와이푸드 양꼬치고기 램500g 쯔란30g 증정,"[식품·건강, 정육·계란류, 수입육·기타, 호주산]",16800,1,5.0,http://www.tmon.co.kr/deal/1648136462\n,1


In [8]:
#상품 리스트
prod_id_ls = list(review_data['prod_idx'].unique())
len(prod_id_ls)

9834

In [9]:
#유저 리스트
member_id_ls = list(review_data['memberSrl'].unique())
len(member_id_ls)

58322

In [10]:
#상품X유저 데이터프레임 생성(index = 상품ID, columns = 유저ID)
df = pd.DataFrame(index=prod_id_ls, columns=member_id_ls)

In [11]:
#상품X유저 데이터프레임에 유저별 상품만족도 삽입
for mem_id in member_id_ls :
    is_memberSrl = review_data['memberSrl'] == mem_id
    mem_prod_ls = list(review_data[is_memberSrl]['prod_idx'])
    mem_satis_ls = list(review_data[is_memberSrl]['prod_satis'])
    for i,j in zip(mem_prod_ls,mem_satis_ls) :
        df[mem_id][i] = j

In [13]:
df = df.fillna(0)

In [14]:
#생성된 데이터프레임 df를 통해 모델링 진행 
df

,101994422,5016629,298123,80235,17873545,70477358,21627189,20400529,10205573,2707693,...,61043278,64185254,12447653,376355,35356381,73024842,5851257,41814697,40265865,49325577
2865723582,5,5,5,5,5,5,4,4,5,5,...,0,0,0,0,0,0,0,0,0,0
512851998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
760761146,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
212815213,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3030712126,0,0,0,0,0,0,4,0,0,5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2764649526,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3518409054,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3173194022,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4292984610,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 상품 기반 

In [85]:
adj_matrix = df.values # 상품 기반

* 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [92]:
target_prod_id, target_prod_vector = 0, adj_matrix[0] #타겟 상품을 특정

In [96]:
temp_prod_id = prod_id_ls[target_prod_id]
print('target_prod_id : {}'.format(temp_prod_id))
print('target_prod_title : {}'.format(prod_data[prod_data['prod_idx'] == temp_prod_id]['title']))

target_prod_id : 2865723582
target_prod_title : 0    [컬러푸드] 선착순20%할인쿠폰+해남 후기갑 꿀고구마 중상1kg 정품\n한입/특상/...
Name: title, dtype: object


In [95]:
best_match, best_match_id, best_match_vector = 9999, -1, [] #초기화

for prod_id, prod_vector in enumerate(adj_matrix) :
    if target_prod_id != prod_id : #타겟 상품과 다른 상품일 경우 유사도 측정
        euclidean_dist = np.sqrt(np.sum(np.square(target_prod_vector - prod_vector))) #유클리드 거리 계산
        if euclidean_dist < best_match : #유사도가 높은 것으로 best_match 갱신
            best_match = euclidean_dist
            best_match_id = prod_id
            best_match_vector = prod_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 327.6232592475693, Best Match ID : 5654


In [97]:
#To-Do fix : 추천 상품 갯수 더 늘릴 것
recommend_prod_id = prod_id_ls[best_match_id]
print('target_prod_id : {}'.format(recommend_prod_id))
print('target_prod_title : {}'.format(prod_data[prod_data['prod_idx'] == recommend_prod_id]['title']))

target_prod_id : 3502657066
target_prod_title : 15706    매콤한 감성 낙곱새 500g 1팩 /감칠맛 소스 증정
Name: title, dtype: object


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [98]:
def compute_cos_similarity(v1, v2) :
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1, v2)
    return dot / (norm1 * norm2)

In [175]:
target_prod_id, target_prod_vector = 0, adj_matrix[0] #타겟 상품을 특정

In [176]:
temp_prod_id = prod_id_ls[target_prod_id]
print('target_prod_id : {}'.format(temp_prod_id))
print('target_prod_title : {}'.format(prod_data[prod_data['prod_idx'] == temp_prod_id]['title']))

target_prod_id : 2865723582
target_prod_title : 0    [컬러푸드] 선착순20%할인쿠폰+해남 후기갑 꿀고구마 중상1kg 정품\n한입/특상/...
Name: title, dtype: object


In [177]:
best_match, best_match_id, best_match_vector = -1, -1, [] #초기화

for prod_id, prod_vector in enumerate(adj_matrix) :
    if target_prod_id != prod_id : #타겟 상품과 다른 상품일 경우 유사도 측정
        cosine_similarity = compute_cos_similarity(target_prod_vector, prod_vector) #코사인 유사도 계산
        if cosine_similarity > best_match : #유사도가 높은 것으로 best_match 갱신
            best_match = cosine_similarity
            best_match_id = prod_id
            best_match_vector = prod_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 0.37967140750126216, Best Match ID : 570


In [178]:
#To-Do fix : 추천 상품 갯수 더 늘릴 것
recommend_prod_id = prod_id_ls[best_match_id]
print('target_prod_id : {}'.format(recommend_prod_id))
print('target_prod_title : {}'.format(prod_data[prod_data['prod_idx'] == recommend_prod_id]['title']))

target_prod_id : 814420358
target_prod_title : 1302    [맛있닭] 닭가슴살 도시락/스테이크/스팀/볼 외 71종 5팩 골라담기
Name: title, dtype: object


### 사용자 기반

In [102]:
adj_matrix = adj_matrix.T #사용자 기반

In [84]:
my_id, my_vector = 0, adj_matrix[0] #내 아이디를 특정
print('my_id : {}'.format(member_id_ls[my_id]))
print('my_id가 구매한 상품 목록')

my_id, my_vector = 0, adj_matrix[0]
a = list(np.where(my_vector > 0)[0])

recommend_prod_list = []
for prod_idx in a :
    recommend_prod_list.append(prod_id_ls[prod_idx])

recommend_prod_titles = prod_data[prod_data['prod_idx'].isin(recommend_prod_list)]['title']
recommend_prod_titles

my_id : 101994422
my_id가 구매한 상품 목록


0        [컬러푸드] 선착순20%할인쿠폰+해남 후기갑 꿀고구마 중상1kg 정품\n한입/특상/...
5             [컬러푸드] 봉팔형님 국내산 햇 밤꿀고구마 & 호박고구마 3kg 5kg 10kg
1266             [퍼스트위크] 티몬블랙딜 통통떡볶이1-2인분 외 순대/튀김/어묵/핫바 모음
6041                               2020 씨없는 청도반시 5KG 60과이상
18207    [퍼스트위크] 필수특가 개당 175원 롯데 청춘간식 키스틱 20개+사면살수록 늘어나...
20665                                [무료배송] 농심 새우깡 90g 10봉
Name: title, dtype: object

In [42]:
best_match, best_match_id, best_match_vector = 9999, -1, [] #초기화

for user_id, user_vector in enumerate(adj_matrix) :
    if my_id != user_id : #내 아이디와 다르다면 유사도 측정
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match : #유사도가 높은 것으로 best_match 갱신
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 12.206555615733702, Best Match ID : 1651


In [43]:
#내가 보진 않았지만, best_match_id로 선정된 user가 산 상품들 추천
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)) :
    log1, log2 = log #log는 (1, 0) 형식의 튜플
    if log1 < 1. and log2 > 0. : #나느 보지않았고(1이 아님) best_match_id는 본 상품들을 리스트에 삽입 
        recommend_list.append(i)
print(recommend_list)

[2564, 4851, 5878]


In [44]:
recommend_prod_list = []
for prod_idx in recommend_list :
    recommend_prod_list.append(prod_id_ls[prod_idx])

In [45]:
#추천 리스트 
recommend_prod_titles = prod_data[prod_data['prod_idx'].isin(recommend_prod_list)]['title']
recommend_prod_titles

6062               [퍼스트위크] 무한타임 고소한 호두 500g+500g외 견과류 건강세일
14750    [퍼스트위크] 무한타임 국내산 햇 세세멸치 1kg 박스/옵션2개 구매시 멸치100g 증정
18022    [퍼스트위크] 무한타임 맛기차콘 10gX20개 외 추억의 불량식품/수입과자 64종 ...
Name: title, dtype: object

In [38]:
def compute_cos_similarity(v1, v2) :
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1, v2)
    return dot / (norm1 * norm2)

In [39]:
my_id, my_vector = 0, adj_matrix[0] #내 아이디를 특정
best_match, best_match_id, best_match_vector = -1, -1, [] #초기화

for user_id, user_vector in enumerate(adj_matrix) :
    if my_id != user_id : #내 아이디와 다르다면 유사도 측정
        cosine_similarity = compute_cos_similarity(my_vector, user_vector)
        if cosine_similarity > best_match : #유사도가 높은 것으로 best_match 갱신
            best_match = cosine_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 0.37967140750126216, Best Match ID : 570


In [34]:
#내가 보진 않았지만, best_match_id로 선정된 user가 본 상품들 추천
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)) :
    log1, log2 = log #log는 (1, 0) 형식의 튜플
    if log1 < 1. and log2 > 0. : #나느 보지않았고(1이 아님) best_match_id는 본 상품들을 리스트에 삽입 
        recommend_list.append(i)
print(recommend_list)

[4, 20, 532, 3900]


In [40]:
recommend_prod_list = []
for prod_idx in recommend_list :
    recommend_prod_list.append(prod_id_ls[prod_idx])

In [41]:
#추천 리스트 
recommend_prod_titles = prod_data[prod_data['prod_idx'].isin(recommend_prod_list)]['title']
recommend_prod_titles

4        [봉팔형님] 밤꿀고구마 중 1kg 외 꿀고구마 4종 크기별 모음 / 2개 구매시 3...
20       [퍼스트위크] 티몬블랙딜 봉팔형님 햇 밤꿀고구마 중 1kg 외 꿀고구마 4종 크기별...
1267                        [무료배송] 홍코너 떡볶이 1+1+1+1+1 /총 5봉
10311    [퍼스트위크] 무한타임 오일가게 이태리 직수입 올리브유 포도씨유 해바라기씨유 250...
Name: title, dtype: object

In [103]:
adj_matrix

array([[5, 0, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [104]:
#U :사용자, S :특이값 벡터, V : 아이템
U, S, V = randomized_svd(adj_matrix, n_components =2 )
S = np.diag(S)

In [142]:
print(U.shape)
print(S.shape)
print(V.shape)

(58322, 2)
(2, 2)
(2, 9834)


In [124]:
np.matmul(np.matmul(U,S), V) #복원된 인접행렬

array([[2.02435407e+00, 3.07905875e-02, 2.13847754e-02, ...,
        5.68679963e-04, 1.47990543e-03, 4.52472785e-04],
       [2.13996783e+00, 3.47814314e-02, 2.48915635e-02, ...,
        6.70562488e-04, 1.63054059e-03, 4.92017964e-04],
       [5.40767338e+00, 9.16877758e-02, 6.67865190e-02, ...,
        1.81250700e-03, 4.23276866e-03, 1.26662322e-03],
       ...,
       [5.14308979e-02, 9.12393572e-04, 6.76525391e-04, ...,
        1.84935676e-05, 4.14525287e-05, 1.22943654e-05],
       [2.92482057e-02, 5.07993571e-04, 3.73599047e-04, ...,
        1.01789896e-05, 2.32515284e-05, 6.92491653e-06],
       [1.56081585e-01, 3.26326664e-03, 2.55922217e-03, ...,
        7.14934612e-05, 1.40440622e-04, 4.03454139e-05]])

In [137]:
#생성된 U를 활용
target_id_idx = 20 #내 아이디를 특정
my_id, my_vector = target_id_idx, U[target_id_idx] 
print('my_id : {}'.format(member_id_ls[my_id]))
print('my_id가 구매한 상품 목록')

a = list(np.where(my_vector > 0)[0])

recommend_prod_list = []
for prod_idx in a :
    recommend_prod_list.append(prod_id_ls[prod_idx])

recommend_prod_titles = prod_data[prod_data['prod_idx'].isin(recommend_prod_list)]['title']
recommend_prod_titles

my_id : 347179
my_id가 구매한 상품 목록


0        [컬러푸드] 선착순20%할인쿠폰+해남 후기갑 꿀고구마 중상1kg 정품\n한입/특상/...
1215           [퍼스트위크] 티몬블랙딜 쉐프330 오리지널 떡볶이 2인분 외 7종+사은품증정
1261               [퍼스트위크] 필수특가 삼시세떡 국물떡볶이 320g 외 야끼만두 김말이
1437     [퍼스트위크] 티몬블랙딜 천일 볶음밥 10봉+떡갈비1봉 골라담기/야채/김치/새우/라...
18192                [퍼스트위크] 무한타임 수입과자가게 사탕 초콜릿 과자 스낵 113종
Name: title, dtype: object

In [148]:
#생성된 U를 활용
best_match, best_match_id, best_match_vector = -1, -1, [] #초기화

for user_id, user_vector in enumerate(U) :
    if my_id != user_id : #내 아이디와 다르다면 유사도 측정
        cosine_similarity = compute_cos_similarity(my_vector, user_vector)
        if cosine_similarity > best_match : #유사도가 높은 것으로 best_match 갱신
            best_match = cosine_similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 0.9999999999983664, Best Match ID : 2091


In [158]:
#내가 보진 않았지만, best_match_id로 선정된 user가 본 상품들 추천
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])) :
    log1, log2 = log #log는 (1, 0) 형식의 튜플
    if log1 < 1. and log2 > 0. : #나느 보지않았고(1이 아님) best_match_id는 본 상품들을 리스트에 삽입 
        recommend_list.append(i)
print(recommend_list)

[7, 21, 27, 31, 165, 637, 1205, 2533, 4823, 8621]


In [159]:
recommend_prod_list = []
for prod_idx in recommend_list :
    recommend_prod_list.append(prod_id_ls[prod_idx])

In [160]:
#추천 리스트 
recommend_prod_titles = prod_data[prod_data['prod_idx'].isin(recommend_prod_list)]['title']
recommend_prod_titles

7                                샐러드미인 샐러드/푸딩 1kg 1+1 골라담기
21       [퍼스트위크] 무한타임 그린투유 양파농가돕기 19년 무안 햇양파 특 5kg/햇 수미...
27                 [트래블경북] 20년 포슬포슬 수미감자 10kg,20kg/대용량/식당용
31       냉동야채16종모음 브로콜리 컬리플라워 그린빈 오크라 미니양배추 혼합야채 미니당근 단...
175                           국내산 창녕 남해 햇 깐마늘 5kg (중소/중/대)
1366              [퍼스트위크] 티몬블랙딜 구어유 1+1+1 1kg/닭날개/닭다리 소스2덤
1971                      [신일푸드팜] 엄지식품 납작당면군만두 야끼만두 1.400g
6031                              성주 참외 대용량 11kg /소과/중과/대과
14722                 [퍼스트위크] 무한타임 국내산 세세멸치 1kg/ 다시멸치/ 밥새우
27831                   국산 현미쌀 2/4/10kg (무농약현미,유기농현미,발아현미)
Name: title, dtype: object

- 상품기반 추천

In [171]:
target_prod_idx = 100 #타겟 상품 특정
target_prod_id, target_prod_vector = target_prod_idx, V.T[target_prod_idx] # 항목기반. V는 곱으로 decompositon 해줬기때문에 트랜스포즈 필요

In [172]:
temp_prod_id = prod_id_ls[target_prod_id]
print('target_prod_id : {}'.format(temp_prod_id))
print('target_prod_title : {}'.format(prod_data[prod_data['prod_idx'] == temp_prod_id]['title']))

target_prod_id : 4262926378
target_prod_title : 102    2020년 수확 포슬포슬 황토 산앤들 햇수미감자 4.5kg 중대과 /2BOX 구매시...
Name: title, dtype: object


In [173]:
#생성된 V를 활용
best_match, best_match_id, best_match_vector = -1, -1, [] #초기화

for prod_id, prod_vector in enumerate(V.T) :
    if target_prod_id != prod_id : #내 아이디와 다르다면 유사도 측정
        cosine_similarity = compute_cos_similarity(target_prod_vector, prod_vector)
        if cosine_similarity > best_match : #유사도가 높은 것으로 best_match 갱신
            best_match = cosine_similarity
            best_match_id = prod_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 0.9999999955292285, Best Match ID : 2986


In [174]:
#To-Do fix : 추천 상품 갯수 더 늘릴 것
recommend_prod_id = prod_id_ls[best_match_id]
print('target_prod_id : {}'.format(recommend_prod_id))
print('target_prod_title : {}'.format(prod_data[prod_data['prod_idx'] == recommend_prod_id]['title']))

target_prod_id : 648232986
target_prod_title : 7346    [대천김] 최신제조 대천김 모음전 무료배송
Name: title, dtype: object


## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [1]:
!pip install scikit-surprise
#설치 안될시
#1. 터미널에 conda install -c conda-forge scikit-surprise 입력
#2. PackagesNotFoundError: The following packages are not available from current channels: 에러발생시
#2-1. conda config --append channels conda-forge 입력 후 다시 시도

In [192]:
from surprise import KNNBasic, SVD, SVDpp, NMF, BaselineOnly, KNNBasic, KNNWithMeans, KNNWithZScore, CoClustering, NormalPredictor, SlopeOne
from surprise.accuracy import rmse, mae
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader

In [180]:
review_data[['memberSrl', 'prod_idx', 'prod_satis']]

,memberSrl,prod_idx,prod_satis
0,101994422,2865723582,5
1,5016629,2865723582,5
2,298123,2865723582,5
3,80235,2865723582,5
4,17873545,2865723582,5
...,...,...,...
568705,153566,3873456926,5
568706,22123157,3873456926,5
568707,80018294,3873456926,5
568709,21201529,3873456926,5


In [182]:
#pandas 데이터 프레임으로부터 데이터셋을 로딩하기 위해, load_from_df() 메소드 사용
#Reader object와 rating_scale 파라미터 특정
#데이터 프레임의 형태는 사용자 id, 아이템 id, 평가
reader = Reader(rating_scale=(0, 9))
data = Dataset.load_from_df(review_data[['memberSrl', 'prod_idx', 'prod_satis']], reader)

In [200]:
benchmark = []
#rmse(Root Mean Squared Error), MAE(Mean Absolute Error)
for model in [SVD(), NMF(), BaselineOnly(), CoClustering()] :
    results = cross_validate(model, data, measures=['rmse', 'mae'], cv = 5, verbose = False)
    
    temp = pd.DataFrame.from_dict(results).mean(axis = 0)
    temp = temp.append(pd.Series([str(model).split('.')[-1]], index =['Model']))
    benchmark.append(temp)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


KeyError: "None of ['model'] are in the columns"

In [215]:
surprise_results = pd.DataFrame(benchmark).set_index('Model').sort_values('test_rmse')

In [216]:
surprise_results

,test_rmse,test_mae,fit_time,test_time
Model,,,,
SVD,0.367415,0.289661,25.587984,1.085115
CoClustering,0.368986,0.243414,18.426157,0.853321
BaselineOnly,0.375273,0.311638,2.394998,0.540774
NMF,0.509330,0.426607,33.571784,0.923151


In [185]:
model = SVD()
cross_validate(model, data, measures=['rmse', 'mae'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3664  0.3683  0.3660  0.3683  0.3680  0.3674  0.0010  
MAE (testset)     0.2889  0.2909  0.2884  0.2904  0.2903  0.2898  0.0009  
Fit time          25.22   25.44   24.87   24.65   25.28   25.09   0.29    
Test time         0.83    0.84    1.22    0.78    1.31    0.99    0.22    


{'test_rmse': array([0.36638339, 0.36832695, 0.36599199, 0.36829674, 0.36800351]),
 'test_mae': array([0.28893044, 0.29090415, 0.28843439, 0.29039475, 0.29031798]),
 'fit_time': (25.215593814849854,
  25.43500828742981,
  24.869486093521118,
  24.646082639694214,
  25.27839231491089),
 'test_time': (0.8277878761291504,
  0.840752124786377,
  1.2187750339508057,
  0.7759263515472412,
  1.3075480461120605)}

In [189]:
#메모리 문제
# model = KNNBasic()
# cross_validate(model, data, measures=['rmse', 'mae'], cv = 5, n_jobs =4, verbose = True)

In [190]:
model = NMF()
cross_validate(model, data, measures=['rmse', 'mae'], cv = 5, n_jobs =4, verbose = True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.5081  0.5104  0.5121  0.5090  0.5080  0.5095  0.0015  
MAE (testset)     0.4256  0.4268  0.4301  0.4271  0.4259  0.4271  0.0016  
Fit time          39.51   43.32   43.33   40.77   35.09   40.40   3.04    
Test time         1.27    1.24    1.14    0.89    0.88    1.08    0.17    


{'test_rmse': array([0.50809714, 0.5103615 , 0.51206417, 0.50901538, 0.50800927]),
 'test_mae': array([0.42563666, 0.42681899, 0.43010389, 0.42711685, 0.42585516]),
 'fit_time': (39.50639200210571,
  43.31521010398865,
  43.32518434524536,
  40.7690167427063,
  35.08920073509216),
 'test_time': (1.2696053981781006,
  1.2386889457702637,
  1.1389541625976562,
  0.8896465301513672,
  0.8786842823028564)}

In [194]:
model = BaselineOnly()
cross_validate(model, data, measures=['rmse', 'mae'], cv = 5, n_jobs =4, verbose = True)

Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3742  0.3752  0.3772  0.3750  0.3748  0.3753  0.0010  
MAE (testset)     0.3109  0.3116  0.3130  0.3115  0.3111  0.3116  0.0007  
Fit time          0.40    0.40    0.39    0.51    0.40    0.42    0.04    
Test time         0.67    0.76    0.77    0.73    0.82    0.75    0.05    


{'test_rmse': array([0.37416734, 0.37520402, 0.37719627, 0.37498961, 0.37478233]),
 'test_mae': array([0.3109002 , 0.31156994, 0.31301829, 0.31145378, 0.3111352 ]),
 'fit_time': (0.3999321460723877,
  0.4029550552368164,
  0.38995838165283203,
  0.5056493282318115,
  0.39797115325927734),
 'test_time': (0.6692483425140381,
  0.7619295120239258,
  0.7689447402954102,
  0.7320435047149658,
  0.8237967491149902)}

In [195]:
model = CoClustering()
cross_validate(model, data, measures=['rmse', 'mae'], cv = 5, n_jobs =4, verbose = True)

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3676  0.3686  0.3694  0.3671  0.3698  0.3685  0.0010  
MAE (testset)     0.2410  0.2420  0.2396  0.2407  0.2383  0.2403  0.0013  
Fit time          14.24   15.45   15.25   13.85   12.47   14.26   1.08    
Test time         1.01    0.96    0.95    0.77    0.63    0.86    0.14    


{'test_rmse': array([0.36760859, 0.36859797, 0.36936582, 0.36707556, 0.36983522]),
 'test_mae': array([0.24096807, 0.24197854, 0.23961312, 0.24072646, 0.23827581]),
 'fit_time': (14.244921684265137,
  15.453688621520996,
  15.254220724105835,
  13.85396409034729,
  12.469709634780884),
 'test_time': (1.0053105354309082,
  0.9554471969604492,
  0.9484648704528809,
  0.7679483890533447,
  0.6313140392303467)}

In [229]:
from surprise.model_selection import train_test_split
from surprise import accuracy

trainset, testset = train_test_split(data, test_size = 0.25)
model = BaselineOnly()
predictions = model.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.3774


0.37737504608480443

In [243]:
#To-Do fix : 모델 확정 후 저장

In [238]:
#유저가 평가한 아이템 수
def get_Iu(user_id) :
    try :
        return len(trainset.ur[trainset.to_inner_uid(user_id)])
    except ValueError :
        return 0
    
#아이템을 평가한 유저 수
def get_Ui(item_id) :
    try :
        return len(trainset.ir[trainset.to_inner_iid(item_id)])
    except ValueError :
        return 0
    
df = pd.DataFrame(predictions, columns = ['user_id','item_id', 'rating', 'est', 'details'])
df['Iu'] = df.user_id.apply(get_Iu)
df['Ui'] = df.item_id.apply(get_Ui)
df['err'] = abs(df.est - df.rating)

In [239]:
df

,user_id,item_id,rating,est,details,Iu,Ui,err
0,1493282,2297962462,5.0,4.847037,{'was_impossible': False},26,174,0.152963
1,90251414,4262797202,5.0,4.766824,{'was_impossible': False},5,396,0.233176
2,2089266,218855365,5.0,4.692943,{'was_impossible': False},5,605,0.307057
3,33860373,4347430018,5.0,4.736367,{'was_impossible': False},2,43,0.263633
4,28801125,2635339642,5.0,4.435223,{'was_impossible': False},12,60,0.564777
...,...,...,...,...,...,...,...,...
139061,86200234,2853302034,5.0,4.858362,{'was_impossible': False},5,179,0.141638
139062,20386189,3422248238,5.0,4.751357,{'was_impossible': False},7,313,0.248643
139063,479079,814420358,5.0,4.791974,{'was_impossible': False},5,25,0.208026
139064,91667054,2726665850,4.0,4.772392,{'was_impossible': False},6,5,0.772392


In [240]:
best_predictions = df.sort_values(by = 'err')[:10]
worst_predictions = df.sort_values(by = 'err')[-10:]

In [241]:
best_predictions

,user_id,item_id,rating,est,details,Iu,Ui,err
42058,45487797,1565742962,5.0,5.000073,{'was_impossible': False},11,27,0.000073
121938,59875986,2461240282,5.0,4.999918,{'was_impossible': False},22,12,0.000082
66211,7739577,2361728674,5.0,5.000126,{'was_impossible': False},27,35,0.000126
19085,1506954,2707159158,5.0,5.000189,{'was_impossible': False},23,41,0.000189
39475,23322901,2001455378,5.0,4.999776,{'was_impossible': False},17,48,0.000224
127061,526323,3307399846,5.0,5.000445,{'was_impossible': False},67,53,0.000445
28073,87189730,3764111394,5.0,5.000457,{'was_impossible': False},16,40,0.000457
95819,33239905,236644317,5.0,5.000498,{'was_impossible': False},19,119,0.000498
48851,17951285,3054457222,5.0,4.999442,{'was_impossible': False},35,139,0.000558
40333,33369545,3097278118,5.0,5.000616,{'was_impossible': False},45,17,0.000616


In [242]:
worst_predictions

,user_id,item_id,rating,est,details,Iu,Ui,err
125973,23253113,4397222310,4.0,4.977759,{'was_impossible': False},27,28,0.977759
92583,39142201,2813248998,4.0,4.977962,{'was_impossible': False},25,148,0.977962
72800,87309082,4355224450,4.0,4.978562,{'was_impossible': False},17,181,0.978562
36813,29026445,2966243526,4.0,4.978709,{'was_impossible': False},41,22,0.978709
66299,36310105,3554510742,4.0,4.989363,{'was_impossible': False},11,31,0.989363
38815,526323,1099550674,4.0,5.000565,{'was_impossible': False},67,5,1.000565
33998,47123645,748426326,4.0,5.004331,{'was_impossible': False},28,60,1.004331
28022,97240130,2637414306,4.0,5.017195,{'was_impossible': False},28,95,1.017195
113497,3439161,2240152278,4.0,5.027161,{'was_impossible': False},22,13,1.027161
68481,33244805,1444592130,4.0,5.041206,{'was_impossible': False},26,16,1.041206


In [ ]:
#To-Do fix : 3 이하 만족도 전처리된 데이터말고 풀 데이터로 적용
#-> 